## A MultiServer Queue

Let's develop a simulation for single FIFO Queue that leads to 3 servers.  
 - Customer arrivals follow a poisson process with rate $\lambda$, (i.e. the interarrival times are exponentially distributed with mean $1/\lambda$).
 - Service Times are follow an exponential distribution with mean $1/\mu$.
 - There are $c$ different servers.

 We want to simulate one replication path with $100$ customers.

In [1]:
import numpy as np  

In [2]:
def sim_one_path(lam, mu, c, numCustomers):
    #This information will help us track the "state" of the simulation
    #create the c different queues and keep track of when the the current customer finishes service
    servers = [np.inf] * c
    customers_in_service = [-1] * c #this will track which customer is being served by each server, -1 means no customer is being served
    queue = [] #the queue starts empty.  It will contain a list of the customers currently waiting.
    next_arrival = np.random.exponential(1/lam)

    #this is where we will log outputs of simulation
    #we will track the arrival time of each customer, when they start service, and when they depart service
    A = np.zeros(numCustomers)
    S = np.zeros(numCustomers)
    D = np.zeros(numCustomers)
    A[0] = next_arrival

    now = 0
    indx_arriving_customer = 0

    while True:
        #fast forward time untl the next "event" happens
        next_service_completion = np.min(servers)
        if next_arrival < next_service_completion:
            now = next_arrival
            
            #if there's an empty server, customer enters service
            if np.max(servers) == np.inf:
                indx_empty_server = np.argmax(servers)
                servers[indx_empty_server] = now + np.random.exponential(1/mu)
                customers_in_service[indx_empty_server] = indx_arriving_customer
                
                #log the start of service
                if indx_arriving_customer < numCustomers:
                    S[indx_arriving_customer] = now

            else: #all servers busy, customer enters queue.
                queue.append(indx_arriving_customer)

            next_arrival = now + np.random.exponential(1/lam)
            indx_arriving_customer += 1

            #only log the arrival if it's in the first num_customers
            if indx_arriving_customer < numCustomers:
                #log the arrival time
                A[indx_arriving_customer] = next_arrival

        else: #the next event is a service completion
            now = next_service_completion
            indx_completing_server = np.argmin(servers)

            #log the departure time if its in teh first numCustomers
            leaving_customer = customers_in_service[indx_completing_server]
            if leaving_customer < numCustomers:
                D[leaving_customer] = now


            #if there are customers waiting, the next customer enters service
            if len(queue) > 0:
                servers[indx_completing_server] = now + np.random.exponential(1/mu)
                next_customer = queue[0] 
                queue = queue[1:] #remove the first customer from the queue
                customers_in_service[indx_completing_server] = next_customer

                #log the start of service for the next customer if it's within first numCustomers
                if next_customer < numCustomers:
                    S[next_customer] = now
                
            else: #no customers waiting, server is now empty
                servers[indx_completing_server] = np.inf
                customers_in_service[indx_completing_server] = -1
            
        #need to check if the first numCustomer arrivals have all been processed
        if np.min(D[:numCustomers]) > 0:
            break
    return A, S, D

In [3]:
A, S, D = sim_one_path(1, .8, 2, 30)

#print an array with columns A, S, D, with entries rounded to two digits
output = np.vstack((A, S, D)).T
print("Arrival, Start, Departure")
print(np.round(output, 2))


Arrival, Start, Departure
[[ 1.5   1.5   1.6 ]
 [ 2.02  2.02  2.47]
 [ 4.31  4.31  5.22]
 [ 5.74  5.74  5.91]
 [ 6.19  6.19  8.33]
 [ 6.19  6.19  7.29]
 [ 6.91  7.29  7.31]
 [ 7.4   7.4  10.47]
 [ 7.69  8.33 10.11]
 [ 7.89 10.11 10.2 ]
 [ 8.14 10.2  11.08]
 [ 9.09 10.47 11.34]
 [ 9.66 11.08 14.88]
 [ 9.97 11.34 11.71]
 [10.92 11.71 12.17]
 [12.57 12.57 13.75]
 [15.57 15.57 16.07]
 [16.48 16.48 16.97]
 [17.23 17.23 20.99]
 [18.45 18.45 19.03]
 [19.36 19.36 20.08]
 [19.78 20.08 23.66]
 [20.26 20.99 22.62]
 [21.87 22.62 23.26]
 [24.17 24.17 26.1 ]
 [26.7  26.7  27.51]
 [29.31 29.31 30.63]
 [30.36 30.36 32.08]
 [30.51 30.63 30.88]
 [31.59 31.59 31.67]]


#### For you:  Write a function that takes in the arrays A, S, D and computes 
 - The average waiting time across customers
 - The average length of the queue (averaged over time)

In [4]:
avg_waiting_time = np.mean(S-A)
print(f"Average waiting time: {avg_waiting_time:.2f}")

def avg_queue_length(numDiscretization_points = 100):
    #create an equispaced time grid between 0 and the last departure
    time_grid = np.linspace(0, np.max(D), numDiscretization_points)
    #calculate teh queue length at each time on the time grid
    queue_length = np.zeros(numDiscretization_points)
    for i, t in enumerate(time_grid):
        #count how many customers are in the queue at time t
        queue_length[i] = np.sum((A <= t) & (D > t))
    #return the average queue length
    return np.mean(queue_length)
avg_length = avg_queue_length()
print(f"Average queue length: {avg_length:.2f}")


Average waiting time: 0.41
Average queue length: 1.44


### (For you): Compute the expected average waiting time across the first 100 arrivals using a simulation with 1000 path

In [5]:
exp_avg_waiting = 0.
for rep in range(1000):
    A, S, D = sim_one_path(1, .8, 2, 30)
    exp_avg_waiting += np.mean(S - A)

exp_avg_waiting /= 1000
print(f"Expected average waiting time: {exp_avg_waiting:.2f}")


Expected average waiting time: 0.57


### More interesting service times
Suppose you believe that 10\% of customers have very difficult requests that take a long time for service, say they take $1/\mu_2 > 1/\mu_1$ time.  How would you alter your simulation?

In [6]:
def gen_service_distribution(mu_fast, mu_slow, p_slow):
    #flip acoin with probability p_slow to determine if the service time is fast or slow
    if np.random.rand() < p_slow:
        return np.random.exponential(1/mu_slow)
    else:
        return np.random.exponential(1/mu_fast)
    
#if you read the follwoing code, it's identical to the other simulation except where we simulate servie times we call above function!
#said differently, we only needed to change a few lines of our simulation to handle potentially slow customers!
def sim_one_path_with_mixed_service(lam, mu_fast, mu_slow, p_slow, c, numCustomers):    
    #This information will help us track the "state" of the simulation
    #create the c different queues and keep track of when the the current customer finishes service
    servers = [np.inf] * c
    customers_in_service = [-1] * c #this will track which customer is being served by each server, -1 means no customer is being served
    queue = [] #the queue starts empty.  It will contain a list of the customers currently waiting.
    next_arrival = np.random.exponential(1/lam)

    #this is where we will log outputs of simulation
    #we will track the arrival time of each customer, when they start service, and when they depart service
    A = np.zeros(numCustomers)
    S = np.zeros(numCustomers)
    D = np.zeros(numCustomers)
    A[0] = next_arrival

    now = 0
    indx_arriving_customer = 0

    while True:
        #fast forward time untl the next "event" happens
        next_service_completion = np.min(servers)
        if next_arrival < next_service_completion:
            now = next_arrival
            
            #if there's an empty server, customer enters service
            if np.max(servers) == np.inf:
                indx_empty_server = np.argmax(servers)
                #servers[indx_empty_server] = now + np.random.exponential(1/mu)  #changed to below
                servers[indx_empty_server] = now + gen_service_distribution(mu_fast, mu_slow, p_slow)
                customers_in_service[indx_empty_server] = indx_arriving_customer
                
                #log the start of service
                if indx_arriving_customer < numCustomers:
                    S[indx_arriving_customer] = now

            else: #all servers busy, customer enters queue.
                queue.append(indx_arriving_customer)

            next_arrival = now + np.random.exponential(1/lam)
            indx_arriving_customer += 1

            #only log the arrival if it's in the first num_customers
            if indx_arriving_customer < numCustomers:
                #log the arrival time
                A[indx_arriving_customer] = next_arrival

        else: #the next event is a service completion
            now = next_service_completion
            indx_completing_server = np.argmin(servers)

            #log the departure time if its in teh first numCustomers
            leaving_customer = customers_in_service[indx_completing_server]
            if leaving_customer < numCustomers:
                D[leaving_customer] = now


            #if there are customers waiting, the next customer enters service
            if len(queue) > 0:
                #servers[indx_completing_server] = now + np.random.exponential(1/mu)  #changed to below
                servers[indx_completing_server] = now + gen_service_distribution(mu_fast, mu_slow, p_slow)
                next_customer = queue[0] 
                queue = queue[1:] #remove the first customer from the queue
                customers_in_service[indx_completing_server] = next_customer

                #log the start of service for the next customer if it's within first numCustomers
                if next_customer < numCustomers:
                    S[next_customer] = now
                
            else: #no customers waiting, server is now empty
                servers[indx_completing_server] = np.inf
                customers_in_service[indx_completing_server] = -1
            
        #need to check if the first numCustomer arrivals have all been processed
        if np.min(D[:numCustomers]) > 0:
            break
    return A, S, D


In [7]:
A, S, D = sim_one_path_with_mixed_service(1, 0.8, 2, 0.2, 2, 50)

#print a matrix with columns A, S, D, with entries rounded to two digits
output = np.vstack((A, S, D)).T
print("Arrival, Start, Departure")
print(np.round(output, 2))

avg_waiting_time = np.mean(S - A)
print(f"Average waiting time with mixed service: {avg_waiting_time:.2f}")

Arrival, Start, Departure
[[ 0.06  0.06  0.82]
 [ 3.18  3.18  3.26]
 [ 6.34  6.34  6.87]
 [ 6.9   6.9  12.99]
 [ 7.43  7.43  7.62]
 [ 9.8   9.8  11.42]
 [ 9.86 11.42 11.82]
 [12.35 12.35 12.36]
 [13.29 13.29 15.14]
 [14.59 14.59 16.1 ]
 [14.64 15.14 17.54]
 [15.45 16.1  16.25]
 [15.59 16.25 18.1 ]
 [15.7  17.54 17.75]
 [16.51 17.75 17.85]
 [19.95 19.95 21.  ]
 [20.16 20.16 22.35]
 [20.5  21.   22.92]
 [21.33 22.35 22.44]
 [21.64 22.44 23.22]
 [23.04 23.04 30.3 ]
 [23.27 23.27 24.75]
 [23.6  24.75 25.38]
 [23.66 25.38 27.71]
 [25.15 27.71 28.2 ]
 [25.22 28.2  29.45]
 [26.02 29.45 29.61]
 [28.78 29.61 30.07]
 [29.14 30.07 30.89]
 [29.25 30.3  30.73]
 [31.01 31.01 31.63]
 [31.09 31.09 31.53]
 [33.55 33.55 34.41]
 [34.65 34.65 35.51]
 [34.75 34.75 37.68]
 [35.52 35.52 36.12]
 [36.81 36.81 37.58]
 [37.62 37.62 38.5 ]
 [37.8  37.8  40.48]
 [38.59 38.59 40.27]
 [38.89 40.27 42.67]
 [39.41 40.48 41.28]
 [42.22 42.22 42.34]
 [42.65 42.65 42.86]
 [42.85 42.85 42.85]
 [42.95 42.95 43.03]
 [45.8  